In [9]:
from nltk import ngrams
from nltk.tokenize import word_tokenize
import unicodedata as ud
import pymongo
import re
import networkx as nx
import pickle
from keyword_extractor import KeywordExtractor
import math
import keywords as kw

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]
with open('../speech_indexes.pickle', 'rb') as handle:
    speech_indexes = pickle.load(handle)

### Keywords for a single document

In [ ]:
ex = KeywordExtractor()
with open('../stopwords.txt', encoding='utf-8') as file:
    stopwords = [line.rstrip() for line in file]
    stopwords = set(stopwords)
with open('../domain-specific-stopwords.txt', encoding='utf-8') as file:
    domain_specific_stopwords = [line.rstrip() for line in file]

In [ ]:
#FIND KEYWORDS FOR DOCUMENTS IN INDEX
for i in range(0, len(speech_indexes)):
    print("Doc:", speech_indexes[i])
    speech = list(database.find({"_id": str(speech_indexes[i]) }, { "_id": 0, "speech": 1 }))
    keywords = ex.document_keywords(speech[0]['speech'], stopwords, domain_specific_stopwords, window = 3)
    database.update_one({"_id":str(speech_indexes[i])},{'$set':{'keywords':list(keywords)}})

### Keywords per party

In [3]:
parties = database.distinct("political_party")

In [5]:
keywords_by_party = kw.keywords_per_party(parties, database)

ανεξαρτητοι (εκτος κομματος)
Party ανεξαρτητοι (εκτος κομματος)
ανεξαρτητοι δημοκρατικοι βουλευτες
Party ανεξαρτητοι δημοκρατικοι βουλευτες
ανεξαρτητοι ελληνες - πανος καμμενος
Party ανεξαρτητοι ελληνες - πανος καμμενος
ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια
Party ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια
αντιπολιτευση
Party αντιπολιτευση
βουλη
Party βουλη
δημοκρατικη ανανεωση
Party δημοκρατικη ανανεωση
δημοκρατικη αριστερα
Party δημοκρατικη αριστερα
δημοκρατικη συμπαραταξη (πανελληνιο σοσιαλιστικο κινημα - δημοκρατικη αριστερα)
Party δημοκρατικη συμπαραταξη (πανελληνιο σοσιαλιστικο κινημα - δημοκρατικη αριστερα)
δημοκρατικο κοινωνικο κινημα
Party δημοκρατικο κοινωνικο κινημα
ελληνικη λυση - κυριακος βελοπουλος
Party ελληνικη λυση - κυριακος βελοπουλος
ενωση κεντρωων
Party ενωση κεντρωων
εξωκοινοβουλευτικός
Party εξωκοινοβουλευτικός
κινημα αλλαγης
Party κινημα αλλαγης
κομμουνιστικο κομμα ελλαδας
Party κομμουνιστικο κομμα ελλαδας
λαικη ενοτητα
Party 

In [6]:
with open('keywords_by_party.pickle', 'wb') as handle:
    pickle.dump(keywords_by_party, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
print(keywords_by_party['συνασπισμος ριζοσπαστικης αριστερας'])
print(keywords_by_party['ελληνικη λυση - κυριακος βελοπουλος'])
print(keywords_by_party['νεα δημοκρατια'])
print(keywords_by_party['ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια'])
print(keywords_by_party['ενωση κεντρωων'])

{'δημοκρατ': 4994, 'νε': 4927, 'τροπολογ': 2902, 'συριζ': 2291, 'ευρ': 2235, 'συνεδριας': 1965, 'σχεδ': 1944, 'ανθρωπ': 1943, 'αναπτυξ': 1925, 'εργας': 1894, 'υγει': 1867, 'συστημ': 1847, 'αρχ': 1836, 'σωμ': 1826, 'δικαιωμ': 1823, 'ερωτης': 1768, 'εργαζομεν': 1736, 'μετρ': 1698}
{'δημοκρατ': 263, 'νε': 251, 'συριζ': 164, 'λυς': 140, 'ευρ': 137, 'κυκλ': 111, 'ανθρωπ': 111, 'συνεδριας': 89, 'ωρ': 85, 'ερωτης': 83, 'σωμ': 81, 'παιδ': 78, 'πρωθυπουργ': 73, 'κοινοβουλευτ': 73}
{'νε': 13265, 'δημοκρατ': 12138, 'τροπολογ': 7789, 'εθν': 7509, 'αρχ': 7014, 'αναπτυξ': 6460, 'ερωτης': 6266, 'σχεδ': 6120, 'κοινοβουλευτ': 5582, 'ευρ': 5278, 'συνεδριας': 5205, 'σωμ': 5108, 'αντιπολιτευς': 4893, 'υπηρες': 4716, 'στοιχει': 4629, 'ρυθμις': 4609, 'εκπροσωπ': 4550, 'τομ': 4541, 'δικαιωμ': 4489}
{'δημοκρατ': 298, 'νε': 276, 'εθν': 170, 'ανθρωπ': 158, 'τροπολογ': 149, 'ευρ': 145, 'εκατομμυρ': 143, 'ερωτης': 135, 'τραπεζ': 133, 'αναπτυξ': 125, 'αντιπολιτευς': 121, 'αμυν': 112, 'ανεξαρτητ': 111, 'δισεκατομμυ

### Keywords per parliament member

In [3]:
#get distinct parliament member names in a list
member_names = database.distinct("member_name")
#years = database.distinct("year")

In [4]:
keywords_by_member = kw.keywords_per_member(member_names, database)

Member αβδελας κωνσταντινου αποστολος
Member αβραμακης αντωνιου ελευθεριος
Member αβραμιδης νικολαου γαβριηλ
Member αβραμοπουλος λαμπρου δημητριος
Member αγαθοπουλου δημητριου ειρηνη-ελενη
Member αγατσα αριστειδη αριαδνη
Member αγγελης στεφανου ανεστης
Member αγγελοπουλος κωνσταντινου νικολαος
Member αγγελουσης αθανασιου ευσταθιος
Member αγγουρακης γεωργιου χαραλαμπος
Member αγοραστος αναστασιου κωνσταντινος
Member αδαμοπουλος αθανασιου γεωργιος
Member αδαμοπουλου αθανασιου αγγελικη
Member αδαμου στεργιου κωνσταντινα
Member αδρακτας φωτιου παναγιωτης
Member αηδονης ευαγγελου χρηστος
Member αθανασιαδης θεοδωρου αλεξανδρος
Member αθανασιος ηλιοπουλος
Member αθανασιος μικρουτσικος
Member αθανασιος σκορδας
Member αθανασιος τσαυταρης
Member αθανασιου ευαγγελου μαρια
Member αθανασιου ευθυμιου αθανασιος (νασος)
Member αθανασιου χριστοφα χαραλαμπος
Member αθανασοπουλος ανδρεα νικολαος
Member αιβαλιωτης ιωαννη κωνσταντινος
Member αιβατιδης γεωργιου ιωαννης
Member αικατερινη νοτοπουλου
Member ακ

In [5]:
print(keywords_by_member['αβδελας κωνσταντινου αποστολος'])

{'κυκλ': 111, 'συνεδριας': 88, 'σωμ': 71, 'ερωτης': 69, 'δυτικ': 57, 'κοινοβουλευτ': 55, 'επικαιρ ': 50, 'επικαιρ ερωτης ': 48}


In [7]:
import pickle

with open('member_keywords.pickle', 'wb') as handle:
    pickle.dump(keywords_by_member, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('member_keywords.pickle', 'rb') as handle:
#    mm = pickle.load(handle)


### Keywords per member per government

In [2]:
#get distinct member names
members = database.distinct("member_name")
governments_per_member = {}
#get governments for each member
for member in members:
    print(f"Member {member}")
    governments_per_member[member] = {}
    pipeline = [{'$match' : {'member_name':member}}, {'$group':{'_id':'$government'}}]
    governments = database.aggregate(pipeline)
    for government in governments:
        keywords = kw.get_member_keywords(member, database, government=government['_id'])
        governments_per_member[member][government['_id']] = keywords

Member nan
Member αβδελας κωνσταντινου αποστολος
Member αβραμακης αντωνιου ελευθεριος
Member αβραμιδης νικολαου γαβριηλ
Member αβραμοπουλος λαμπρου δημητριος
Member αγαθοπουλου δημητριου ειρηνη-ελενη
Member αγατσα αριστειδη αριαδνη
Member αγγελης στεφανου ανεστης
Member αγγελοπουλος κωνσταντινου νικολαος
Member αγγελουσης αθανασιου ευσταθιος
Member αγγουρακης γεωργιου χαραλαμπος
Member αγοραστος αναστασιου κωνσταντινος
Member αδαμοπουλος αθανασιου γεωργιος
Member αδαμοπουλου αθανασιου αγγελικη
Member αδαμου στεργιου κωνσταντινα
Member αδρακτας φωτιου παναγιωτης
Member αηδονης ευαγγελου χρηστος
Member αθανασιαδης θεοδωρου αλεξανδρος
Member αθανασιος ηλιοπουλος
Member αθανασιος μικρουτσικος
Member αθανασιος σκορδας
Member αθανασιος τσαυταρης
Member αθανασιου ευαγγελου μαρια
Member αθανασιου ευθυμιου αθανασιος (νασος)
Member αθανασιου χριστοφα χαραλαμπος
Member αθανασοπουλος ανδρεα νικολαος
Member αιβαλιωτης ιωαννη κωνσταντινος
Member αιβατιδης γεωργιου ιωαννης
Member αικατερινη νοτοπουλο

In [3]:
import pickle

with open('keywords_per_member_per_government.pickle', 'wb') as handle:
    pickle.dump(governments_per_member, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
governments_per_member['αθανασιος τσαυταρης']

{"['σαμαρα κ. αντωνιου(21/06/2012-26/01/2015)']": {'γεωργ': 52,
  'αγροτ': 47,
  'ευρ': 37,
  'παραγωγ': 36,
  'προϊοντα': 36,
  'τοπ': 34,
  'κτηνοτροφ': 32}}

In [2]:
import pickle
with open('keywords_per_member_per_government.pickle', 'rb') as handle:
    keywords_per_member_per_government = pickle.load(handle)

In [11]:
govs = list(database.distinct("government"))

In [12]:
print(govs[1])

['ζολωτα ξενοφωντα(23/11/1989-11/04/1990)']


In [12]:
total = list(keywords_per_member_per_government.values())
print(list(total[123].values()))

[{'τροπολογ': 2, 'ρυθμις': 2, 'συμβουλ': 2}, {'αναπτυξ': 1, 'προγραμμ': 1}]


In [10]:
import plotly.graph_objects as go

animals=list(total[123].keys())

fig = go.Figure(data=[
    go.Bar(name=list(total[123].keys())[0], x=animals, y=[20, 14, 23]),
    go.Bar(name=list(total[123].keys())[1], x=animals, y=[12, 18, 29])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()